Worth taking a look at:
- [HeartPy](https://python-heart-rate-analysis-toolkit.readthedocs.io/en/latest/) library
- [Biosppy](https://biosppy.readthedocs.io/en/stable/index.html) library
Generally useful:
- [Neurokit2](https://neurokit2.readthedocs.io/en/latest/) library

# Imports

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import glob
from tqdm import tqdm
from typing import Dict
from scipy.signal import resample
import neurokit2 as nk
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import butter, filtfilt
import heartpy as hp
import seaborn as sns
import numpy as np
from biosppy.signals import ecg
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler

In [2]:
%matplotlib inline
sns.set_theme()

# Import Data

Used Dataset:
- [WESAD: Multimodal Dataset for Wearable Stress and Affect Detection](https://ubicomp.eti.uni-siegen.de/home/datasets/icmi18/): Exclusively importing the available pickle files from each subject (S2.pkl, S2.pkl, ...) and combining them
- Self-Generated Dataset (ECGPrivate):
    - Device: Bioplux Chest (and Finger) Band
    - Dataframes are modeled following the already implemented pydantic models

In [3]:
def dict_to_dataframe_WESAD(dict):
    """
    Convert a dictionary object loaded from the WESAD dataset to a pandas DataFrames. The WESAD dataset contains multiple dictionary objects, each containing a single subject's data. See https://archive.ics.uci.edu/ml/datasets/WESAD+%28Wearable+Stress+and+Affect+Detection%29 for more information on the WESAD dataset.

    :param dict: A dictionary object containing the data to convert.
    :type dict: dict

    :return: A pandas DataFrame containing the converted data.
    :rtype: pandas DataFrame
    """

    df = pd.DataFrame()

    df["ECG"] = dict["signal"]["chest"]["ECG"].flatten()
    df["label"] = dict["label"].flatten()
    df["subject"] = dict["subject"]

    df.reset_index(names="timestamp_idx", inplace=True)

    return df

In [4]:
def combine_pickles_to_df(pattern, wesad_ecg=False):
    """
    Load multiple pickle files that contain the same type of data from different subjects or sessions and combining them into a single DataFrame.

    :param pattern: A glob pattern that matches the pickle files to load.
    :type pattern: str
    :param wesad: Whether to convert dictionary objects to DataFrames (only used for WESAD dataset). Defaults to False.
    :type wesad: bool

    :return: A pandas DataFrame that combines the loaded DataFrames.
    :rtype: pandas DataFrame
    """

    dfs = []
    for file_path in tqdm(glob.glob(pattern)):
        with open(file_path, 'rb') as f:
            try:
                df = pd.read_pickle(f)
            except AttributeError:
                raise ValueError(f"Expected a pandas DataFrame, but got {type(df)}")
            if isinstance(df, Dict) and wesad_ecg:
                df = dict_to_dataframe_WESAD(df)
            dfs.append(df)
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

### WESAD Data

In [5]:
# Configs
# wesad respian sampling rate in hz
fs_wesad = 700

In [6]:
%%time
# load WESAD
df_wesad = combine_pickles_to_df('../data/raw/S15.pkl', wesad_ecg=True)

0it [00:00, ?it/s]


ValueError: No objects to concatenate

In [7]:
df_wesad

NameError: name 'df_wesad' is not defined

In [ ]:
# Plot filtered ECG data
plt.figure(figsize=(20, 5))
plt.plot("timestamp_idx", "ECG", data=df_wesad)
plt.title("Raw ECG Data")
plt.xlabel("timestamp_idx")
plt.ylabel("voltage")
plt.xlim(10000, 30000)
plt.show()

### Self-generated dataset

# Preprocessing

Steps Preprocessing (def preprocess_ecg) ~ process_ecg neurokit2:
1. **Filtering**: Highpass filter (remove baseline wander) and Bandpass filter (remove noise) using the [Scipy](https://www.scipy.org/) library (butterworth filter)
2. **Resampling**: Resample using the [Scipy](https://www.scipy.org/) library
3. **Scaling/Normalizing**: Standardization (z-score normalization) using the [Scikit-learn](https://scikit-learn.org/stable/) library
(summarize previous steps via ecg_clean by neurokit2)
4. **R-Peak Detection**: Using the [Neurokit2](https://neurokit2.readthedocs.io/en/latest/) library
2. **Heart Rate Variability**: Using the

## Normalize

**Applied and implemented techniques:**
- **a) Standardization**: z-score normalization (subtracting mean and dividing by standard deviation)

**Reasoning:**
Standardization is a common preprocessing step. It is used to normalize the data so that the data has a mean of 0 and a standard deviation of 1. Normalizing ECG (Electrocardiogram) data is beneficial for several reasons:

1. **Scale invariance:** ECG signals are typically measured in millivolts (mV) and have varying amplitude ranges. By normalizing the data, you can ensure that the features are scaled consistently across different ECG recordings. This helps in making the algorithm or model more robust and independent of the specific scale of the data.

2. **Minimization of bias:** Normalizing the ECG data helps prevent certain features from dominating the analysis simply because of their larger magnitudes. Without normalization, features with larger values can have a disproportionate influence on the overall analysis, potentially leading to biased results.

3. **Facilitation of feature comparison:** Normalizing the ECG data allows for fairer comparisons between different features or channels. It ensures that the features are on a comparable scale, enabling meaningful comparisons based on their values or statistical properties.

4. **Preservation of relative patterns:** ECG waveforms often contain important temporal patterns and relationships. Normalization ensures that these relative patterns are preserved, even if the absolute amplitudes are scaled uniformly. This is crucial for accurate interpretation and analysis of ECG signals.

In [ ]:
# create a StandardScaler object
scaler = StandardScaler()
# Normalize ECG data
normalized_ecg = scaler.fit_transform(df_wesad[["ECG"]])
normalized_df = df_wesad.copy()
normalized_df["ECG"] = normalized_ecg

For further usage we are only going to use the standardized data.

In [ ]:
df = normalized_df.copy()

## Filtering / Denoising

**Applied and implemented techniques:**
- **a) Highpass filter**: Remove baseline wander
- **b) Bandpass filter**: Remove noise

**Reasoning:**
A band-pass filter allows a range of frequencies to pass while attenuating frequencies outside the range. In most cases, a band-pass filter includes both high-pass and low-pass filters as a part of its design. Therefore, a band-pass filter can attenuate both baseline wander and higher frequency noise in one step.

However, in some cases, using a high-pass filter separately before applying a band-pass filter may be necessary to avoid over-filtering and loss of important ECG signal components. This is especially true if the ECG signal contains low-frequency components of interest, such as the T wave or P wave. For this reason both are used. See [here](https://en.wikipedia.org/wiki/Band-pass_filter) for more information on band-pass filters. See [here](https://en.wikipedia.org/wiki/High-pass_filter) for more information on high-pass filters.

a) Remove basline wander via **highpass filter**

Explanation of parameters used for applying a high-pass filter:
- sampling_rate: The sampling rate of the ECG signal in Hz. The sampling rate is the number of samples obtained per second. For example, if the sampling rate is 360 Hz, then 360 samples are obtained per second. The sampling rate is used to determine the cutoff frequency of the high-pass filter.
- cutoff_freq: The cutoff frequency of the high-pass filter in Hz. The cutoff frequency is the frequency below which all frequencies are attenuated.
- nyquist_freq: The Nyquist frequency of the high-pass filter in Hz. The Nyquist frequency is half of the sampling rate (in Hz). For example, if the sampling rate is 360 Hz, then the Nyquist frequency is 180 Hz. The Nyquist frequency is used to determine the order of the high-pass filter. See [here](https://en.wikipedia.org/wiki/Nyquist_frequency) for more information on Nyquist Frequency and the Nyquist-Shannon Sampling Theorem (Nyquist Sampling Rate) which states that a signal can be perfectly reconstructed from its samples if the sampling rate is greater than twice the highest frequency component in the signal. This is why the Nyquist Frequency is half the sampling rate.
- filter_order: The order of the high-pass filter (e.g. 3rd order Butterworth filter). The higher the order, the steeper the filter roll-off (i.e. the faster the filter attenuation). However, higher order filters are more computationally expensive. A 3rd order filter is a good compromise between filter roll-off and computational cost.


In [ ]:
def remove_basline_wander(data, sampling_rate=360, cutoff_freq=0.05):
    """
    Remove baseline wander from ECG data using a high-pass filter. The high-pass filter will remove all frequencies below the cutoff frequency. The cutoff frequency should be set to the lowest frequency that is still considered baseline wander and not part of the ECG signal. For example, baseline wander is typically between 0.05 Hz and 0.5 Hz. Therefore, a cutoff frequency of 0.05 Hz is a good starting point. However, if the ECG signal contains low-frequency components of interest, such as the T wave or P wave, then a higher cutoff frequency may be necessary to avoid over-filtering and loss of important ECG signal components. For this reason, a cutoff frequency of 0.5 Hz is also a good starting point. The cutoff frequency can be adjusted as needed. See https://en.wikipedia.org/wiki/High-pass_filter for more information on high-pass filters.

    :param data: ECG data as a 1-dimensional numpy array
    :type data: numpy array
    :param sampling_rate: sampling rate of ECG data (Hz), defaults to 360
    :type sampling_rate: int, optional
    :param cutoff_freq: cutoff frequency of high-pass filter (Hz), defaults to 0.05
    :type cutoff_freq: float, optional

    :return: ECG data with baseline wander removed
    :rtype: numpy array
    """

    # Define filter parameters
    # Nyquist frequency - The highest frequency that can be represented given the sampling frequency. Nyquist Frequency is half the sampling rate (in Hz).
    nyquist_freq = 0.5 * sampling_rate
    # Filter order - The higher the order, the steeper the filter roll-off (i.e. the more aggressive the filter is at removing frequencies outside the passband).
    filter_order = 3
    # Apply high-pass filter
    b, a = butter(filter_order, cutoff_freq/nyquist_freq, 'highpass')
    filtered_data = filtfilt(b, a, data)

    return filtered_data

In [ ]:
filtered_ecg = remove_basline_wander(df["ECG"].values, sampling_rate=fs_wesad)
filtered_df = df.copy()
filtered_df["ECG"] = pd.Series(filtered_ecg)

In [ ]:
# Plot filtered ECG data
plt.figure(figsize=(20, 5))
plt.plot("timestamp_idx", "ECG", data=df, label="no filter")
plt.plot("timestamp_idx", "ECG", data=filtered_df, label="highpass filter (Butterworth)")
plt.title("Filtered ECG Data Comparison")
plt.xlabel("timestamp_idx")
plt.ylabel("voltage")
plt.xlim(20000, 24000)
plt.legend()
plt.show()

b) Remove noise via **bandpass filter**

In [ ]:
def remove_noise(data, sampling_rate=360, lowcut=0.5, highcut=45):
    """
    Remove noise from ECG data using a band-pass filter. The band-pass filter will remove all frequencies below the lowcut frequency and above the highcut frequency. The lowcut frequency should be set to the lowest frequency that is still considered noise and not part of the ECG signal. For example, noise is typically between 0.5 Hz and 45 Hz. Therefore, a lowcut frequency of 0.5 Hz is a good starting point. However, if the ECG signal contains low-frequency components of interest, such as the T wave or P wave, then a higher lowcut frequency may be necessary to avoid over-filtering and loss of important ECG signal components. For this reason, a lowcut frequency of 5 Hz is also a good starting point. The lowcut frequency can be adjusted as needed. The highcut frequency should be set to the highest frequency that is still considered noise and not part of the ECG signal. For example, noise is typically between 0.5 Hz and 45 Hz. Therefore, a highcut frequency of 45 Hz is a good starting point. However, if the ECG signal contains high-frequency components of interest, such as the QRS complex, then a lower highcut frequency may be necessary to avoid over-filtering and loss of important ECG signal components. For this reason, a highcut frequency of 15 Hz is also a good starting point. The highcut frequency can be adjusted as needed. See https://en.wikipedia.org/wiki/Band-pass_filter for more information on band-pass filters.

    :param data: ECG data as a 1-dimensional numpy array
    :type data: numpy array
    :param sampling_rate: sampling rate of ECG data (Hz), defaults to 360
    :type sampling_rate: int, optional
    :param lowcut: lowcut frequency of band-pass filter (Hz), defaults to 0.5
    :type lowcut: float, optional
    :param highcut: highcut frequency of band-pass filter (Hz), defaults to 45
    :type highcut: float, optional

    :return: ECG data with noise removed
    :rtype: numpy array
    """

    # Define filter parameters
    nyquist_freq = 0.5 * sampling_rate
    # Define cutoff frequencies (remove all frequencies below lowcut and above highcut)
    low = lowcut / nyquist_freq
    high = highcut / nyquist_freq
    # Initialize filter
    b, a = butter(4, [low, high], btype='band')
    # Apply filter twice (combined filter) to remove forward and reverse phase shift. See https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.filtfilt.html for more information on filtfilt.
    filtered_data = filtfilt(b, a, data)

    return filtered_data

In [ ]:
filtered_ecg = remove_noise(df["ECG"].values, sampling_rate=fs_wesad)
filtered_df = df.copy()
filtered_df["ECG"] = pd.Series(filtered_ecg)

In [ ]:
# Plot filtered ECG data
plt.figure(figsize=(20, 5))
plt.plot("timestamp_idx", "ECG", data=df, label="no filter")
plt.plot("timestamp_idx", "ECG", data=filtered_df, label="banpass filter (Butterworth)")
plt.title("Filtered ECG Data Comparison")
plt.xlabel("timestamp_idx")
plt.ylabel("voltage")
plt.xlim(20000, 24000)
plt.legend()
plt.show()

## Sample Rate
Change sampling rate

In [ ]:
def resample_data(data, original_sampling_rate, new_sampling_rate=360):
    # Resample ECG data to a new sampling rate
    resampled_data = resample(data, int(len(data) * new_sampling_rate / original_sampling_rate))
    return resampled_data

In [ ]:
resampled_ecg = resample_data(df["ECG"], original_sampling_rate=fs_wesad)
resampled_df = df.copy()
resampled_df["ECG"] = pd.Series(resampled_ecg)

In [ ]:
# Plot resampled ECG data
plt.figure(figsize=(20, 5))
plt.plot("timestamp_idx", "ECG", data=df, label=f"{fs_wesad} HZ")
plt.plot("timestamp_idx", "ECG", data=resampled_df, label="360 HZ")
plt.title("Resampled ECG Data Comparison")
plt.xlabel("timestamp_idx")
plt.ylabel("voltage")
plt.xlim(20000, 24000)
plt.legend()
plt.show()

## Preprocess via **pantompkins** and neurokit

In [ ]:
# Process the ecg signal
cleaned = nk.ecg_clean(df['ECG'], sampling_rate=fs_wesad, method="pantompkins1985")
signals, info = nk.ecg_peaks(cleaned, sampling_rate=fs_wesad, method="pantompkins1985", correct_artifacts=True)

# sum up
#nk_preprocessed_ecg = nk.ecg_process(df['ECG'], sampling_rate=fs_wesad, method="pantompkins1985")

# Plot the data if of interest
nk.events_plot(info["ECG_R_Peaks"], cleaned)

# Preprocessing Pipeline

In [ ]:
def preprocess_ecg(data, sampling_rate, original_sampling_rate=700, new_sampling_rate=360):

    # Remove basline wander using highpass filter
    filtered_data = remove_basline_wander(data=data, sampling_rate=sampling_rate)
    # Remove noise from ECG data using bandpass filter
    filtered_data = remove_noise(data=filtered_data, sampling_rate=sampling_rate)
    # Resample ECG data to a new sampling rate
    if new_sampling_rate is not None and new_sampling_rate != original_sampling_rate:
        filtered_data = resample(filtered_data, int(len(filtered_data) * new_sampling_rate / original_sampling_rate))
    # Normalize ECG data to have zero mean and unit variance
    scaler = StandardScaler()
    normalized_data = scaler.fit_transform(filtered_data.reshape(-1, 1)).reshape(-1)

    return normalized_data

In [ ]:
%%time
preprocessed_ecg = preprocess_ecg(df_wesad["ECG"].values, original_sampling_rate=fs_wesad)
preprocessed_df = df_wesad.copy()
preprocessed_df["ECG"] = pd.Series(preprocessed_ecg)

# Feature Extraction
Sources:
- "A brief review on the analysis of cardiovascular signals" by S. Sanei and J. A. Chambers (2007), "A guide to artefact detection in ECG signals" by S. S. Makkar et al. (2011)
- "A robust algorithm for heart beats detection using ECG signals" by M. S. Al-Angari and S. A. Sahakian (2007)
- "A comparison of heart rate variability analysis using smartphone and traditional RR-interval meters" by J. H. Shin et al. (2013), "Heart rate variability: a review" by M. Malik et al. (1996)
- "Heart rate variability: from bench to bedside" by G. B. Carrick-Ranson et al. (2015)
- "Heart rate variability: clinical applications" by M. Malik et al. (1996)
- "Heart rate variability: standards of measurement, physiological interpretation and clinical use" by the Task Force of the European Society of Cardiology and the North American Society of Pacing and Electrophysiology (1996)
- "A brief review on the analysis of cardiovascular signals" by S. Sanei and J. A. Chambers (2007)
- "A guide to artefact detection in ECG signals" by S. S. Makkar et al. (2011)
- "A robust algorithm for heart beats detection using ECG signals" by M. S. Al-Angari and S. A. Sahakian (2007)
- "A comparison of heart rate variability analysis using smartphone and traditional RR-interval meters" by J. H. Shin et al. (2013)
- "Heart rate variability: a review" by M. Malik et al. (1996)
- "Heart rate variability: from bench to bedside" by G. B. Carrick-Ranson et al. (2015)
- "Heart rate variability: clinical applications" by M. Malik et al. (1996)

* **Time Domain Features**:
    * **Mean RR interval**: the average length of time between successive R peaks in the ECG signal. This feature provides a general estimate of the heart rate.
    * **SDNN (standard deviation of normal-to-normal intervals)**: the standard deviation of the intervals between successive R peaks in the ECG signal, excluding any abnormal beats or ectopic beats. This feature provides an estimate of the overall variability of the heart rate.
    * **RMSSD (root mean square of successive differences)**: the square root of the mean of the squared differences between successive normal-to-normal intervals. This feature provides an estimate of the short-term variability of the heart rate.
    * **NN50**: the number of pairs of successive normal-to-normal intervals that differ by more than 50 ms. This feature provides an estimate of the parasympathetic nervous system activity.
    * **pNN50**: the percentage of pairs of successive normal-to-normal intervals that differ by more than 50 ms. This feature provides a normalized measure of the parasympathetic nervous system activity.
* **Frequency Domain Feature**:
    * **LF (low-frequency) band power**: the power in the ECG signal in the frequency band between 0.04 Hz and 0.15 Hz. This feature provides an estimate of the sympathetic and parasympathetic nervous system activity.
    * **HF (high-frequency) band power**: the power in the ECG signal in the frequency band between 0.15 Hz and 0.4 Hz. This feature provides an estimate of the parasympathetic nervous system activity.
    * **LF/HF ratio**: the ratio of LF power to HF power. This feature provides an estimate of the balance between sympathetic and parasympathetic nervous system activity.
* nonlinear features from RR intervals? - no

In [ ]:
def hrv_features(data, fs):
    # Compute RR intervals from ECG data
    rpeaks, = ecg.hamilton_segmenter(signal=data, sampling_rate=fs)
    rr_intervals = np.diff(rpeaks) / fs

    # Compute time domain features from RR intervals
    mean_rr = np.mean(rr_intervals)
    sdnn = np.std(rr_intervals)
    rmssd = np.sqrt(np.mean(np.square(np.diff(rr_intervals))))
    nn50 = np.sum(np.abs(np.diff(rr_intervals)) > 0.05)
    pnn50 = nn50 / len(rr_intervals)

    # Compute frequency domain features from RR intervals
    freq, power = welch(rr_intervals, fs=fs, nperseg=256, noverlap=128, nfft=512)
    lf_band = np.sum(power[(freq >= 0.04) & (freq <= 0.15)])
    hf_band = np.sum(power[(freq >= 0.15) & (freq <= 0.4)])
    lf_hf_ratio = lf_band / hf_band

    # Return HRV features as a dictionary
    features = {
        'mean_rr': mean_rr,
        'sdnn': sdnn,
        'rmssd': rmssd,
        'nn50': nn50,
        'pnn50': pnn50,
        'lf_band': lf_band,
        'hf_band': hf_band,
        'lf_hf_ratio': lf_hf_ratio,
    }
    return features

In [ ]:
%%time
# Compute HRV features
hrv = hrv_features(preprocessed_ecg, fs_wesad)

In [ ]:
# Convert from sec (s) to millisec (ms)(sx1000 -> ms)

# Print HRV features
print(f"HRV features in seconds: {hrv}")